In [ ]:
!pip install -q -U bitsandbytes==0.42.0 peft==0.8.2 trl==0.7.10 accelerate==0.27.1 datasets==2.17.0 transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.0 MB/s eta 0:00:00


In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

In [ ]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnd_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id,token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN']
                                             )

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
text = "Quote: Imagination is more"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens = 50)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.

-Albert Einstein

The following is a list of the most popular and most recent articles on the subject of imagination.

<strong>Imagination</strong>

<strong>Imagination</strong>

<strong>


In [ ]:
text = "Quote: Imagination is more"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens = 50)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.

-Albert Einstein

The following is a list of the most popular and most recent articles on the subject of imagination.

<strong>Imagination</strong>

<strong>Imagination</strong>

<strong>


In [ ]:
os.environ['WANDB_DISABLED']="false"

In [ ]:
lora_config = LoraConfig(
    r=8,
    target_modules = ["q_proj","o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type= "CAUSAL_LM",
)

In [ ]:
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
data["train"]['author']

['Oscar Wilde',
 'Marilyn Monroe',
 'Albert Einstein',
 'Frank Zappa',
 'Marcus Tullius Cicero',
 'Bernard M. Baruch',
 'William W. Purkey',
 'Dr. Seuss',
 'Mae West',
 'Mahatma Gandhi',
 'Robert Frost',
 'J.K. Rowling,',
 'Albert Camus',
 'Mark Twain',
 'C.S. Lewis,',
 'Elbert Hubbard',
 'Oscar Wilde',
 'Oscar Wilde',
 'Mahatma Gandhi',
 'Martin Luther King Jr.,',
 'Friedrich Nietzsche,',
 'Stephen Chbosky,',
 'Oscar Wilde,',
 'Ralph Waldo Emerson',
 'Rob Siltanen',
 'Narcotics Anonymous',
 'Marilyn Monroe',
 'H. Jackson Brown Jr.,',
 'Andre Gide,',
 'Bob Marley',
 'J.R.R. Tolkien,',
 'J.K. Rowling,',
 'Jane Austen,',
 'Marilyn Monroe',
 'Albert Einstein',
 'J.K. Rowling,',
 'Mark Twain',
 'John Green,',
 'Oscar Wilde,',
 'Neil Gaiman,',
 'Maurice Switzer,',
 'William Shakespeare,',
 'Mark Twain',
 'Allen Saunders',
 'Bill Keane',
 'Thomas A. Edison',
 'Friedrich Nietzsche',
 'Elie Wiesel',
 'Eleanor Roosevelt',
 'Mark Twain',
 'Gordon A. Eadie',
 'douglas adams,',
 'Pablo Neruda,',
 

In [ ]:
def formatting(example):
  text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
  return [text]

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting,
)

In [ ]:
trainer.train()

Step,Training Loss
1,1.681600
2,0.630700
3,1.023200
4,1.032700
5,0.419800
6,1.231800
7,1.096100
8,0.335800
9,0.566000
10,0.481400


TrainOutput(global_step=100, training_loss=0.14717414206825197, metrics={'train_runtime': 79.6742, 'train_samples_per_second': 5.02, 'train_steps_per_second': 1.255, 'total_flos': 55030401331200.0, 'train_loss': 0.14717414206825197, 'epoch': 66.67})

In [ ]:
text = "Quote: A woman is like tea bag"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens = 50)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: A woman is like tea bag. The more you add to her, the less she is worth.
Author: Nicola Yoon
Book: Everything, Everything
Quote: The most wasted of all days is one without laughter.
Author: Nicolas Chamfort
Quote: The most


In [ ]:
text = "Quote: Imagination is more"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens = 50)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: Imagination is more important than knowledge, for knowledge without imagination is apt to be of no value.
Author: Albert Einstein
Quote: The most wasted of all days is one without laughter.
Author: Nicolas Chamfort
Quote: The most wasted of all days


In [ ]:
text = "Quote: The opposite of love is not hate"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens = 50)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: The opposite of love is not hate, it’s fear. The opposite of knowledge is not ignorance; it’s fear.
Author: Aung San Suu Kyi
Quote: The most wasted of all days is one without laughter.
Author: Nicolas Chamfort
Quote
